# NEXUS tool: case study for the NWSAS basin - least-cost options calculations
In this notebook a case study for the NWSAS basin is covered using the `nexus_tool` package. The energy requirements for agricultural irrigation were previously calculated (check the water demand and energy demand case studies). In this case study, the least-cost options to supply the energy needs for agricultural irrigation are identified.

First import the package by running the following block:

In [390]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [391]:
%autoreload
import nexus_tool

[autoreload of nexus_tool failed: Traceback (most recent call last):
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 450, in superreload
    update_generic(old_obj, new_obj)
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 387, in update_generic
    update(a, b)
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 342, in update_class
    if update_generic(old_obj, new_obj): continue
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 387, in update_generic
    update(a, b)
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packa

In [392]:
file_path = r'nwsas_results/pumping_energy.csv'
df = nexus_tool.read_csv(file_path)

In [393]:
file_path = r'nwsas_data/Fuel_prices.xlsx'
fuel_price = nexus_tool.read_excel(file_path)

In [394]:
nwsas = nexus_tool.Model(df)
nwsas.end_year = 2030
nwsas.start_year = 2016
nwsas.discount_rate = 0.05

In [395]:
nwsas.df = nwsas.df.merge(fuel_price, on='country')

In [396]:
nwsas.create_wind_turbine('Wind power', life=20,
                          om_cost=0.02, capital_cost=1300,
                          efficiency=0.6)

In [397]:
nwsas.create_pv_system('Solar PV', life=15,
                       om_cost=0.01, capital_cost=1140,
                       efficiency=0.7)

In [398]:
nwsas.create_standard_tech('Diesel set', life=10, om_cost=0.1,
                           capital_cost=938, fuel_cost=nwsas.df['Dprice1'],
                           fuel_req=0.095, efficiency=0.27, cf = 0.5,
                           emission_factor=2.7, env_cost=0)

In [399]:
nwsas.create_standard_tech('Grid pump', life=10, om_cost=0.1,
                           capital_cost=845, fuel_cost=nwsas.df['Epricelow'],
                           fuel_req=1, efficiency=0.55, cf = 0.8,
                           emission_factor=0.728, env_cost=0)

In [400]:
nwsas.get_cf(['Wind power', 'Solar PV'])
nwsas.get_installed_capacity('all')
nwsas.get_max_capacity('a')
nwsas.get_lcoe()

In [401]:
comp_vars = {'Algeria':['Diesel set','Solar PV','Wind power'],
            'Tunisia':['Diesel set','Solar PV','Wind power'],
            'Libya':['Grid pump','Solar PV','Wind power']}
nwsas.get_least_cost(technologies = comp_vars, geo_boundary = 'country')

In [402]:
nwsas.get_tech_generation()

In [406]:
nwsas.print_summary()

,Irrigated area (ha),Water demand (Mm3),Water intensity (m3/ha),Energy demand (GWh),Average lcoe ($/kWh),Least-cost technologies
global,,,,,,
global,272336.485036,3248.298013,11927.516846,727.876124,0.65759,"Solar PV, Diesel set, Grid pump"
